In [1]:
## LigftGBMを用いて、着順予測を行うモデルを作成するプログラム。単勝_複勝回収率を計算する。

import lightgbm as lgb
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np
import ast
from sklearn.metrics import precision_recall_curve
import os


def split_date(df, test_size):
    sorted_df = df.sort_values('日付')
    train_size = int(len(sorted_df) * (1 - test_size))
    train = sorted_df.iloc[:train_size]
    test = sorted_df.iloc[train_size:]
    return train, test

# データの読み込み
# data = pd.read_csv('encoded/encoded_data.csv')
data = pd.read_csv('encoded/2022_2023encoded_data.csv')

# 着順を変換
data['着順'] = data['着順'].map(lambda x: 1 if x < 4 else 0)

# 特徴量とターゲットの分割
train, test = split_date(data, 0.3)

X_train = train.drop(['着順', 'オッズ', '人気', '上がり', '走破時間', '通過順'], axis=1)
y_train = train['着順']
X_test = test.drop(['着順', 'オッズ', '人気', '上がり', '走破時間', '通過順'], axis=1)
y_test = test['着順']

# LightGBMデータセットの作成
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test)

# クラス比率の計算（データ分割後の訓練データで計算）
ratio = (y_train == 0).sum() / (y_train == 1).sum()  # 負例 / 正例

# ハイパーパラメータ
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'class_weight': 'balanced',
    'random_state': 100,
    'feature_pre_filter': False,
    'lambda_l1': 4.2560625081811865e-05,
    'lambda_l2': 4.74860278547497,
    'num_leaves': 5,
    'feature_fraction': 0.9520000000000001,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'n_estimators': 1000,
    # 'is_unbalance': True,  # クラス不均衡に対応
    'scale_pos_weight': ratio,  # 負例数/正例数
}

print(f"Negative/Positive ratio: {ratio:.2f}")
print(f"負例数（3着外）: {(y_train == 0).sum()}")
print(f"正例数（3着内）: {(y_train == 1).sum()}")


# モデルの訓練
lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train, y_train)
y_pred_train = lgb_clf.predict_proba(X_train)[:, 1]
y_pred = lgb_clf.predict_proba(X_test)[:, 1]

# モデルの評価
print(roc_auc_score(y_test, y_pred))
# 最適な閾値を探索（Fβスコア最大化）
precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred)
fbeta_scores = (1 + 0.5**2) * (precisions * recalls) / (0.5**2 * precisions + recalls)
best_idx = np.argmax(fbeta_scores)
optimal_threshold = thresholds[best_idx]

# # 閾値探索を回収率最大化で行う
# def calculate_profit(th):
#     bets = y_pred >= th
#     return (win_payments[bets].sum() - len(bets)*100) / (len(bets)*100)
# profits = [calculate_profit(t) for t in thresholds]
# optimal_threshold = thresholds[np.argmax(profits)]

print(f"Optimal Threshold: {optimal_threshold:.4f}")
# ----------------------------------------------------------

# 混同行列の計算（閾値変更版）
TP = ((y_test == 1) & (y_pred >= optimal_threshold)).sum()  # 0.5 → optimal_threshold
FP = ((y_test == 0) & (y_pred >= optimal_threshold)).sum()
TN = ((y_test == 0) & (y_pred < optimal_threshold)).sum()
FN = ((y_test == 1) & (y_pred < optimal_threshold)).sum()
total_cases = len(y_test)

accuracy_TP = TP / total_cases * 100
misclassification_rate_FP = FP / total_cases * 100
accuracy_TN = TN / total_cases * 100
misclassification_rate_FN = FN / total_cases * 100

print("Total cases:", total_cases)
print(f"True positives(実際に3着内で、予測も3着内だったもの): {TP} ({accuracy_TP:.2f}%)")
print(f"False positives(実際は3着外だが、予測では3着内だったもの): {FP} ({misclassification_rate_FP:.2f}%)")
print(f"True negatives(実際に3着外で、予測も3着外だったもの): {TN} ({accuracy_TN:.2f}%)")
print(f"False negatives(実際は3着内だが、予測では3着外だったもの): {FN} ({misclassification_rate_FN:.2f}%)")

# モデルの保存
lgb_clf.booster_.save_model('model/model.txt')

# 特徴量の重要度を取得し表示
importance = lgb_clf.feature_importances_
feature_names = X_train.columns
indices = np.argsort(importance)[::-1]

#重みの判定
for f in range(X_train.shape[1]):
    print(f"{f + 1:2d}) {feature_names[indices[f]]:<30} {importance[indices[f]]}")


# 単勝回収率、複勝回収率の計算（修正版）
def load_payback_data(
    years,
    base_dir='payback',
    encoding='SHIFT-JIS',
    sep=',',
    payout_cols=None
):
    if payout_cols is None:
        # 単勝, 複勝, 枠連, 馬連, ワイド, 馬単, 三連複, 三連単 の8列を想定
        payout_cols = ['単勝','複勝','枠連','馬連','ワイド','馬単','三連複','三連単']

    all_dfs = []
    for year in years:
        file_path = os.path.join(base_dir, f"{year}.csv")

        if not os.path.isfile(file_path):
            print(f"[警告] {file_path} は存在しません。")
            continue

        try:
            # race_idを文字列型で読み込み (科学的表記2.02201E+11対策)
            df = pd.read_csv(
                file_path,
                encoding=encoding,
                header=0,       # 1行目に race_id, 単勝, 複勝, ... がある前提
                sep=sep,
                dtype={'race_id': str},
                on_bad_lines='warn'
            )
        except Exception as e:
            print(f"[エラー] {file_path} の読み込みに失敗: {e}")
            continue

        if df.empty:
            print(f"[情報] {file_path} は空、または有効な行がありません。")
            continue

        # race_id の末尾に .0 が付いていたら削除
        df['race_id'] = df['race_id'].str.replace(r'\.0$', '', regex=True)

        # 払戻し列をPythonリストに変換 (ast.literal_eval)
        for col in payout_cols:
            if col in df.columns:
                df[col] = df[col].apply(
                    lambda x: ast.literal_eval(x) 
                    if pd.notna(x) and x.strip().startswith('[') 
                    else []
                )
            else:
                # ファイルに対象列が無い場合は空リストで埋める
                df[col] = [[] for _ in range(len(df))]

        all_dfs.append(df)

    # すべて読み込んだ後、DataFrameを結合
    if not all_dfs:
        print("[警告] 読み込めるデータがありませんでした。空のDataFrameを返します。")
        return pd.DataFrame()

    betting_data = pd.concat(all_dfs, ignore_index=True)
    # race_id をインデックスに設定
    betting_data.set_index('race_id', inplace=True)
    return betting_data

#rangeは(x,x+1)の場合xのみ読み込み
years = range(2022, 2024)  # 2022年のみ読み込み
betting_data  = load_payback_data(years)

# 予測結果を元に賭ける馬を決定
# betting_horses = {(test.iloc[i]['race_id'], test.iloc[i]['馬番']): y_pred[i] for i in range(len(y_pred)) if y_pred[i] >= optimal_threshold}

# 予測結果を元に賭ける馬を決定（閾値変更）
betting_horses = {
    (test.iloc[i]['race_id'], test.iloc[i]['馬番']): y_pred[i] 
    for i in range(len(y_pred)) 
    if y_pred[i] >= optimal_threshold  # 0.5 → optimal_threshold
}
# # race_idを文字列として統一
# test['race_id'] = test['race_id'].astype(str).str.strip().str.replace('.0', '')
betting_data.index = betting_data.index.astype(str).str.strip()


win_return_amount = 0  # 単勝の回収金額
place_return_amount = 0  # 複勝の回収金額
for (race_id, horse_number) in betting_horses:
    
    race_id = str(int(float(race_id)))
    # 修正箇所2: 馬番変換
    horse_number = str(int(float(horse_number)))
    
    # 修正箇所3: インデックス設定
    # betting_data.index = betting_data.iloc[:, 0].astype(str).str.strip().str.replace('.0', '')
    if race_id in betting_data.index:

        race_data = betting_data.loc[race_id]
        win_data = race_data[0]  # 単勝のデータを取得
        place_data = race_data[1]  # 複勝のデータを取得

        for j in range(0, len(win_data), 2):
            if win_data[j] == horse_number:
                win_return_amount += int(win_data[j + 1].replace(',', ''))

        for j in range(0, len(place_data), 2):
            if place_data[j] == horse_number:
                place_return_amount += int(place_data[j + 1].replace(',', ''))
    # else:
    #     print(f"Race ID {race_id} not found in betting data.")

# 単勝と複勝の回収率を計算
betting_amount = len(betting_horses)
win_return_rate = win_return_amount / betting_amount if betting_amount else 0
place_return_rate = place_return_amount / betting_amount if betting_amount else 0


print("単勝回収率:", win_return_rate)
print("複勝回収率:", place_return_rate)


ModuleNotFoundError: No module named 'lightgbm'